<img src="https://gitlab.inf.unibz.it/earth_observation_public/eurac-openeo-examples/-/raw/main/python/aux_data/eurac_EO.png"
     alt="EURAC EO logo"
     style="float: left; margin-right: 10px; max-width: 300px" />

<img src="https://avatars.githubusercontent.com/u/63704085?s=400&u=5dc6c45b7d457ce6f303fbc612091cb2c133482f&v=4"
     alt="SAR2Cube logo"
     style="float: right; max-width: 200px" />
# SAR2Cube and openEO - Accessing and Analyzing SAR data on-the-fly
Author michele.claus@eurac.edu

Date: 2022/09/29

Updated: 2023/05/15

## Useful links:

SAR2Cube website: https://sar2cube.projects.eurac.edu/

openEO main website: https://openeo.org/

openEO Python Client documentation: https://open-eo.github.io/openeo-python-client/index.html

Getting started guide for openEO with python: https://openeo.org/documentation/1.0/python/

## Topics:
- Login to Eurac's openEO backend containing the SAR2Cube data sets
- Compute the intensity over the area of interest with different Multi Look sizes in SAR and geocoded domain
- Apply the radar mask to remove areas affected by layover, foreshortening or shadow effects

## Import all the libraries and utilities functions
The ones included in the eo_utils.py file are needed for the ipyleaflet interactive map

Please check to have the latest `openeo` library. Please check here for the latest available version: https://pypi.org/project/openeo/

In [ ]:
import os
import openeo
import xarray as xr
import matplotlib.pyplot as plt
try:
    from eo_utils import *
except Exception as e:
    print(e)

## Connect and login

In [ ]:
openeoHost = "https://openeo.eurac.edu"
conn = openeo.connect(openeoHost).authenticate_oidc(client_id="openEO_PKCE")

## SAR processing

We load an openEO collection containing pre-processed Sentinel-1 SLC data. It contains real and imaginary parts of each polarization, the longitude and latitude grids used for geocoding, LIA and DEM used for radar masking.

In [ ]:
conn.describe_collection("SAR2Cube_SInCohMap_S1_L0_117_ASC_SOUTH_TYROL")

## Select the AOI
Use the rectangle selection tool to select the area of interest.

If it does not work, it means that some library is missing and the functions contained in `eo_utils.py` could not be loaded.

Uncomment the line defining `spatial_extent` to use a predefined area instead.

In [ ]:
center = [46.5, 11.4]
zoom = 12

eoMap = openeoMap(center,zoom)
eoMap.map

In [ ]:
bbox = eoMap.getBbox()
print("Coordinates selected from map:","\n west",bbox[0],"\n east",bbox[2],"\n south",bbox[1],"\n north",bbox[3])

## SAR processing


### Load the datacube

In [ ]:
conn = openeo.connect(openeoHost).authenticate_oidc(client_id='openEO_PKCE')

collection      = 'SAR2Cube_SInCohMap_S1_L0_117_ASC_SOUTH_TYROL'
temporal_extent = ['2018-06-01T00:00:00.000Z', '2018-06-30T00:00:00.000Z']
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}

S1_slant_range = conn.load_collection(collection,spatial_extent=spatial_extent,temporal_extent=temporal_extent)

Compute the intensity

In [ ]:
i_VV = S1_slant_range.band('i_VV')
q_VV = S1_slant_range.band('q_VV')
S1_INT = (i_VV**2+q_VV**2)
S1_INT_VV = S1_INT.add_dimension(name="bands",label="VV")

Compute the Multi Look over the intensity and convert from linear to dB.

In [ ]:
range_looks   = 4
azimuth_looks = 19

S1_INT_VV_ML = S1_INT_VV.aggregate_spatial_window(
    boundary="trim",
    size=[azimuth_looks,range_looks],
    reducer="mean")

from openeo.processes import log
S1_INT_VV_ML = S1_INT_VV_ML.apply(lambda x: 10*log(x,base=10))

Compute the same Multi Look over the coordinate grids for geocoding

In [ ]:
lat_lon = S1_slant_range.filter_bands(["grid_lon","grid_lat"])

lat_lon_ML = lat_lon.aggregate_spatial_window(
    boundary="trim",
    size=[azimuth_looks,range_looks],
    reducer="mean")

Compute the Multi Look of Local Incidence Angle and Digital Elevation Model for radar masking

In [ ]:
dem_lia = S1_slant_range.filter_bands(["DEM","LIA"])

dem_lia_ML = dem_lia.aggregate_spatial_window(
    boundary="trim",
    size=[azimuth_looks,range_looks],
    reducer="mean")

Remove the temporal dimension, not required for radar_mask

In [ ]:
dem_lia_ML = dem_lia_ML.reduce_dimension(reducer="mean", dimension="DATE")

Use the `radar_mask` process inside an `apply_dimension` process.

`apply_dimension` is used when we need to apply a process over a specific dimension, similarly to `reduce_dimension`, but the specified dimension is not reduced/removed. Instead, the number of output dimension labels (i.e. bands) can grow or shrink.

Use the list_processes method and search for radar_mask to get more details on I/O of this process.

In [ ]:
conn.list_processes() # Search for radar_mask

The `radar_mask` process is not yet an official openEO process and therefore the syntax to apply it is not streamlined as with the other processes.

In [ ]:
from openeo.rest.datacube import PGNode

radar_mask_node = PGNode("radar_mask",
                         data={"from_parameter": "data"},
                         orbit_direction="ASC",
                         foreshortening_th=0.1,
                         layover_th=2.1)
radar_mask = dem_lia_ML.apply_dimension(process=radar_mask_node, dimension="bands")

Rename the output bands with a meaningful name

In [ ]:
radar_mask = radar_mask.rename_labels(target=["layover","foreshortening","shadow"],dimension="bands")

Merge the intensity and the coordinates into the same datacube

In [ ]:
S1_INT_VV_ML = S1_INT_VV_ML.merge_cubes(lat_lon_ML)

Compute the average over time

In [ ]:
S1_INT_ML_VV_MEAN = S1_INT_VV_ML.reduce_dimension(reducer="mean", dimension="DATE")

Merge also the radar_mask output

In [ ]:
S1_INT_ML_VV_MEAN = S1_INT_ML_VV_MEAN.merge_cubes(radar_mask)

Apply geocoding. We choose 20m resolution for the pixel size and the local UTM zone as projection.

We can choose only from 10, 20 or 60m for resolution, for being able to align the data with Sentinel-2 grid.

This process is not yet an official openEO process and therefore the syntax to apply it is not streamlined as with the other processes.

In [ ]:
args_geocoding = {"resolution":20,"crs":32632}
S1_INT_ML_VV_GEOCODED = S1_INT_ML_VV_MEAN.process("geocode",args_geocoding, data=S1_INT_ML_VV_MEAN)

In [ ]:
S1_INT_ML_VV_GEOCODED_NC = S1_INT_ML_VV_GEOCODED.save_result(format="netCDF")

For jobs requiring to process many dates and a big area in the spatial domain, we need to use batch jobs.

The job will run in the background and when it will be marked as finished you can download the result.

In [ ]:
job = conn.create_job(S1_INT_ML_VV_GEOCODED_NC,title="SAR2Cube_South_Tyrol_radar_mask_ASC")
job_id = job.job_id
print('Batch job created with id: ',job_id)
job.start_job()

Visualize the job status

In [ ]:
job = conn.job(job_id)
job

Create the output folder to store the results:

In [ ]:
import os
result_dir_name = "radar_mask_result"
if not os.path.exists(result_dir_name):
    os.mkdir(result_dir_name)

Once the job is marked as finished, you can download the result.

Either via the download link provided in the following visualization:

In [ ]:
result = job.get_results()
result

Or via python code specifying the target location:

In [ ]:
result.download_files('./radar_mask_ASC/')

Loading the netCDF using xArray

In [ ]:
out = xr.open_dataarray('./radar_mask_ASC/result.nc',decode_coords='all').to_dataset(dim='variable')
out

Visualize the result with matplotlib

In [ ]:
plot_args = {'cmap':'Greys_r','add_colorbar':False}
fig, ax = plt.subplots(1,3,figsize=(20,5))
out.VV.plot.imshow(ax=ax[0],**plot_args, vmin=-30,vmax=0)
ax[0].set_title('VV ASC Intensity [dB]')
out.layover.plot.imshow(ax=ax[1],**plot_args, vmin=0,vmax=1)
ax[1].set_title('Layover ASC mask')
out.shadow.plot.imshow(ax=ax[2],**plot_args, vmin=0,vmax=1)
ax[2].set_title('Shadow ASC mask')
plt.show()